# Predict
This file is used to predict the result with trained model,

load checkpoint from model-result/checkpoint-xxxx,

save result to output/dataset.model.checkpoint.csv

Author: Xiaofeng Zhang

Email: Xiaofzhang1@student.unimelb.edu.au

SID: 1194655


In [1]:
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, AutoTokenizer
import torch
import numpy as np 
import os

In [2]:
from datasets import load_from_disk
rumour = load_from_disk("./data/rumour")

# Predict all checkpoint

In [9]:
'''
predict result with bert-base model
'''
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenize text and truncate sequence to be no longer than maximum input of "Distilber"
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_rumour = rumour.map(preprocess_function, batched=True)
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

respath ="bert-results"
model_names = os.listdir("bert-results")
dataset = "test"

for model_name in model_names:
    modle_path=os.path.join(respath,model_name)
    model = AutoModelForSequenceClassification.from_pretrained(modle_path, num_labels=2)
    model.eval()
    with open ("./output/"+dataset+'.'+respath+'.'+model_name+".csv","w") as f:
        f.write("Id,Predicted\n")# heading 
        for i in range(len(tokenized_rumour[dataset])):
                
                _input = tokenized_rumour[dataset][i]['input_ids']
                _mask = tokenized_rumour[dataset][i]['attention_mask']
                model.to("cuda")
                _input = torch.tensor(_input).unsqueeze(0).to("cuda")
                _mask = torch.tensor(_mask).unsqueeze(0).to("cuda")
                
                y=model(input_ids=_input)
                res = np.argmax(y.logits.detach().cpu().numpy())
                f.write("{0},{1}\n".format(i,res))
                

In [3]:
'''
predict result of distilbert model
'''
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# tokenize text and truncate sequence to be no longer than maximum input of "Distilber"
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_rumour = rumour.map(preprocess_function, batched=True)
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

respath ="distilbert-results_s"
model_names = os.listdir(respath)
dataset = "dev"

for model_name in model_names:
    modle_path=os.path.join(respath,model_name)
    model = AutoModelForSequenceClassification.from_pretrained(modle_path, num_labels=2)
    model.eval()
    with open ("./output/"+dataset+'.'+respath+'.'+model_name+".csv","w") as f:
        f.write("Id,Predicted\n")# heading 
        for i in range(len(tokenized_rumour[dataset])):
                
                _input = tokenized_rumour[dataset][i]['input_ids']
                _mask = tokenized_rumour[dataset][i]['attention_mask']
                model.to("cuda")
                _input = torch.tensor(_input).unsqueeze(0).to("cuda")
                _mask = torch.tensor(_mask).unsqueeze(0).to("cuda")
                
                y=model(input_ids=_input)
                res = np.argmax(y.logits.detach().cpu().numpy())
                f.write("{0},{1}\n".format(i,res))
                

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]